# 第8章: 幻覚を避ける

- [レッスン](#lesson)
- [演習](#exercises)
- [サンプルプレイグラウンド](#example-playground)

## セットアップ

以下のセットアップセルを実行して、APIキーを読み込み、`get_completion`ヘルパー関数を確立してください。

In [ ]:
# Pythonの組み込み正規表現ライブラリをインポート
import re
import boto3
import json

# utilsパッケージからhintsモジュールをインポート
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import hints

# IPythonストアからMODEL_NAME変数を取得
%store -r MODEL_NAME
%store -r AWS_REGION

client = boto3.client('bedrock-runtime',region_name=AWS_REGION)

def get_completion(prompt, system='', prefill=''):
    body = json.dumps(
        {
            "anthropic_version": '',
            "max_tokens": 2000,
            "messages":[
              {"role": "user", "content": prompt},
              {"role": "assistant", "content": prefill}
            ],
            "temperature": 0.0,
            "top_p": 1,
            "system": system
        }
    )
    response = client.invoke_model(body=body, modelId=MODEL_NAME)
    response_body = json.loads(response.get('body').read())

    return response_body.get('content')[0].get('text')

---

## レッスン

悪いニュースがあります: **Claudeは時々「幻覚」を起こし、真実ではない、または正当化されない主張をすることがあります**。良いニュースは、幻覚を最小限に抑えるテクニックがあるということです。
				
以下では、これらのテクニックのいくつかを紹介します。具体的には:
- Claudeに質問に対する答えを知らないと言う選択肢を与える
- 回答する前に証拠を見つけるようClaudeに求める

ただし、**幻覚を避けるには多くの方法があり**、このコースですでに学んだ多くのテクニックも含まれます。Claudeが幻覚を起こした場合は、複数のテクニックを試してClaudeの精度を向上させてください。

### 例

ここでは、一般的な事実に関する知識についての質問があり、**Claudeはできるだけ役立とうとするあまり、いくつかの大きなカバを幻覚で作り出しています**。

In [ ]:
# プロンプト
PROMPT = "史上最も重いカバは誰ですか？"

# Claudeの回答を出力
print(get_completion(PROMPT))

ここで試せる解決策は、「**Claudeに逃げ道を与える**」ことです — 回答を断ることができる、または確実に知っている場合にのみ回答するようClaudeに伝えます。

In [ ]:
# プロンプト
PROMPT = "史上最も重いカバは誰ですか？確実に答えを知っている場合にのみ回答してください。"

# Claudeの回答を出力
print(get_completion(PROMPT))

以下のプロンプトでは、ユーザーの質問にほぼ関連しているが完全には関連していない「ディストラクター情報」を含む長い文書をClaudeに与えています。**プロンプトの助けがなければ、Claudeはディストラクター情報に惑わされ**、2020年5月31日時点のMatterportの加入者数について不正確な「幻覚」の回答をしてしまいます。

**注意:** 次の章で学ぶように、**ベストプラクティスはテキストや文書の後に質問を置くこと**ですが、ここではプロンプトを読みやすくするために上部に置いています。プロンプトセルをダブルクリックして全文を確認してください（非常に長いです！）。

In [ ]:
# プロンプト
PROMPT = """<question>2020年5月31日の正確な日付におけるMatterportの加入者数は何人でしたか？</question>
以下の文書を読んでください。その後、<answer>タグ内に簡潔な数値回答を書いてください。

<document>
</document>"""

# Claudeの回答を出力
print(get_completion(PROMPT))

これをどのように修正すればよいでしょうか？長い文書に対する幻覚を減らす優れた方法は、**まずClaudeに証拠を集めさせることです**。

この場合、**まず関連する引用を抽出し、その引用に基づいて回答するようClaudeに指示します**。ここでそうすることで、Claudeは引用が質問に答えていないことを正しく認識します。

In [ ]:
# プロンプト
PROMPT = """<question>2020年5月31日の正確な日付におけるMatterportの加入者数は何人でしたか？</question>
以下の文書を読んでください。次に、<scratchpad>タグ内で、文書から最も関連性の高い引用を抽出し、それがユーザーの質問に答えているか、または十分な詳細が不足しているかを検討してください。その後、<answer>タグ内に簡潔な数値回答を書いてください。

<document>
</document>"""

# Claudeの回答を出力
print(get_completion(PROMPT))

#### ボーナスレッスン

時々、Claudeの幻覚は回答の`temperature`を下げることで解決できます。temperatureは0から1の間の回答の創造性の測定値で、1が最も予測不可能で標準化されていない状態、0が最も一貫性のある状態です。

temperature 0でClaudeに何かを尋ねると、一般的に繰り返し試行しても、ほぼ決定論的な回答セットが得られます（ただし、完全な決定論は保証されません）。temperature 1（またはその間の段階）でClaudeに何かを尋ねると、より変化に富んだ回答が得られます。temperatureやその他のパラメータについて詳しくは[こちら](https://docs.anthropic.com/claude/reference/messages_post)をご覧ください。

上記のコンテンツを変更せずにレッスンのプロンプトを試したい場合は、レッスンノートブックの一番下にある[**サンプルプレイグラウンド**](#example-playground)をご覧ください。

---

## 演習
- [演習 8.1 - ビヨンセの幻覚](#exercise-81---beyoncé-hallucination)
- [演習 8.2 - 目論見書の幻覚](#exercise-82---prospectus-hallucination)

### 演習 8.1 - ビヨンセの幻覚
Claudeに逃げ道を与えることで、`PROMPT`を修正してClaudeの幻覚の問題を解決してください。（ルネサンスはビヨンセの7枚目のスタジオアルバムで、8枚目ではありません。）

まずセルを実行してClaudeが何を幻覚するか見てから、修正を試みることをお勧めします。

In [ ]:
# プロンプト
PROMPT = "スター・パフォーマーのビヨンセが8枚目のスタジオアルバムをリリースしたのは何年ですか？"

# Claudeの回答を取得
response = get_completion(PROMPT)

# 演習の正解度を評価する関数
def grade_exercise(text):
    contains = bool(
        re.search("残念ながら", text) or
        re.search("わかりません", text) or
        re.search("分かりません", text)
    )
    does_not_contain = not bool(re.search("2022", text))
    return contains and does_not_contain

# Claudeの回答と対応する評価を出力
print(response)
print("\n------------------------------------------ 評価 ------------------------------------------")
print("この演習は正しく解決されました:", grade_exercise(response))

❓ ヒントが欲しい場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_8_1_hint)

### 演習 8.2 - 目論見書の幻覚
引用を求めることで、`PROMPT`を修正してClaudeの幻覚の問題を解決してください。正解は、加入者が49倍に増加したことです。

In [ ]:
# プロンプト
PROMPT = """2018年12月から2022年12月までの間に、Matterportの加入者数はどれだけ増加しましたか？

<document>
</document>"""

# Claudeの回答を取得
response = get_completion(PROMPT)

# 演習の正解度を評価する関数
def grade_exercise(text):
    return bool(re.search("49倍", text))

# Claudeの回答と対応する評価を出力
print(response)
print("\n------------------------------------------ 評価 ------------------------------------------")
print("この演習は正しく解決されました:", grade_exercise(response))

❓ ヒントが欲しい場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_8_2_hint)

### おめでとうございます！

ここまでのすべての演習を解決できたなら、次の章に進む準備ができています。プロンプティングを楽しんでください！

---

## サンプルプレイグラウンド

ここは、このレッスンで示されたプロンプトの例を自由に試し、プロンプトを調整してClaudeの応答がどのように変化するか見ることができる領域です。

In [ ]:
# プロンプト
PROMPT = "史上最も重いカバは誰ですか？"

# Claudeの回答を出力
print(get_completion(PROMPT))

In [ ]:
# プロンプト
PROMPT = "史上最も重いカバは誰ですか？確実に答えを知っている場合にのみ回答してください。"

# Claudeの回答を出力
print(get_completion(PROMPT))

In [ ]:
# プロンプト
PROMPT = """<question>2020年5月31日の正確な日付におけるMatterportの加入者数は何人でしたか？</question>
以下の文書を読んでください。その後、<answer>タグ内に簡潔な数値回答を書いてください。

<document>
</document>"""

# Claudeの回答を出力
print(get_completion(PROMPT))

In [ ]:
# プロンプト
PROMPT = """<question>2020年5月31日の正確な日付におけるMatterportの加入者数は何人でしたか？</question>
以下の文書を読んでください。次に、<scratchpad>タグ内で、文書から最も関連性の高い引用を抽出し、それがユーザーの質問に答えているか、または十分な詳細が不足しているかを検討してください。その後、<answer>タグ内に簡潔な数値回答を書いてください。

<document>
</document>"""

# Claudeの回答を出力
print(get_completion(PROMPT))